# Ensemble Learning and Random Forests: Theory and Practice

This notebook is a summary and practical guide based on Chapter 7, covering:
- Ensemble Learning concepts
- Voting classifiers (hard and soft voting)
- Bagging and pasting techniques
- Out-of-bag evaluation
- Random Patches and Random Subspaces
- Random Forests and Extra-Trees
- Feature importance
- Boosting (AdaBoost and Gradient Boosting)
- Stacking ensembles

It contains explanations, example code, and visualizations using Python and scikit-learn.

## 1. Ensemble Learning Overview

Ensemble Learning aggregates the predictions of multiple models (called weak learners) to create a more accurate and robust predictor (strong learner).

Common ensemble methods include:
- Voting (hard and soft)
- Bagging (Bootstrap Aggregating)
- Boosting
- Stacking

The key idea is to combine diverse models to reduce variance and/or bias.

In [1]:
# Example: Hard voting classifier combining Logistic Regression, Random Forest, and SVM
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X, y = make_moons(n_samples=500, noise=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(probability=True, random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)

voting_clf.fit(X_train, y_train)

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"{clf.__class__.__name__} Accuracy: {accuracy_score(y_test, y_pred):.3f}")

LogisticRegression Accuracy: 0.860
RandomForestClassifier Accuracy: 0.893
SVC Accuracy: 0.886
VotingClassifier Accuracy: 0.900


## 2. Hard Voting vs Soft Voting

- **Hard voting:** Predict class with most votes
- **Soft voting:** Predict class with highest average predicted probability

Soft voting often yields better accuracy since it uses confidence weights.

In [2]:
voting_clf_soft = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft'
)

voting_clf_soft.fit(X_train, y_train)
y_pred_soft = voting_clf_soft.predict(X_test)
print(f"Soft Voting Classifier Accuracy: {accuracy_score(y_test, y_pred_soft):.3f}")

Soft Voting Classifier Accuracy: 0.913


## 3. Bagging and Pasting

- Train multiple base estimators on different random samples of data
- **Bagging:** Sampling with replacement (bootstrap)
- **Pasting:** Sampling without replacement

Aggregates outputs by averaging or majority voting.

In [3]:
# Bagging classifier with Decision Trees
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42),
    n_estimators=500,
    max_samples=100,
    bootstrap=True,
    n_jobs=-1,
    random_state=42
)
bag_clf.fit(X_train, y_train)
y_pred_bag = bag_clf.predict(X_test)
print(f"Bagging Classifier Accuracy: {accuracy_score(y_test, y_pred_bag):.3f}")

Bagging Classifier Accuracy: 0.900


## 4. Out-of-Bag Evaluation

Samples not included in bootstrap samples for a given base estimator are called out-of-bag (OOB) instances.
OOB instances can be used for validation without a separate dataset.

Scikit-learn supports OOB score estimation in BaggingClassifier.

In [4]:
bag_clf_oob = BaggingClassifier(
    DecisionTreeClassifier(random_state=42),
    n_estimators=500,
    bootstrap=True,
    oob_score=True,
    n_jobs=-1,
    random_state=42
)
bag_clf_oob.fit(X_train, y_train)

print(f"OOB Score: {bag_clf_oob.oob_score_:.3f}")

OOB Score: 0.899


## 5. Random Patches and Random Subspaces

- Feature sampling together with instance sampling enhances diversity.
- **Random Patches:** Sample instances and features both (useful for high-dimensional data).
- **Random Subspaces:** Sample features only.

Implemented as parameters in BaggingClassifier.

## 6. Random Forests

An ensemble of decision trees trained via bagging with extra randomness: 
- At each split, consider a random subset of features to find the best split.
- This lowers correlation between trees, reduces variance, and improves generalization.
- More efficient and convenient with `RandomForestClassifier` and `RandomForestRegressor`.

In [5]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.3f}")

Random Forest Accuracy: 0.913


## 7. Extra-Trees

Similar to Random Forest but with even more randomness in choosing split thresholds.
- Very fast to train
- Trades slightly more bias for lower variance

In [6]:
from sklearn.ensemble import ExtraTreesClassifier

et_clf = ExtraTreesClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42)
et_clf.fit(X_train, y_train)
y_pred_et = et_clf.predict(X_test)
print(f"Extra-Trees Accuracy: {accuracy_score(y_test, y_pred_et):.3f}")

Extra-Trees Accuracy: 0.913


## 8. Feature Importance

Random Forest allows estimation of feature importance based on average impurity decrease.

Example: Feature importance on iris dataset.

In [7]:
import numpy as np
import matplotlib.pyplot as plt

iris = load_iris()
rf_clf_full = RandomForestClassifier(n_estimators=500, random_state=42)
rf_clf_full.fit(iris.data, iris.target)
importances = rf_clf_full.feature_importances_

plt.barh(iris.feature_names, importances)
plt.xlabel('Feature Importance')
plt.title('Random Forest Feature Importance (Iris)')
plt.show()

## 9. Boosting

Boosting sequentially trains predictors to correct errors from previous ones.
- AdaBoost: updates instance weights to emphasize hard examples.
- Gradient Boosting: fits new predictors to residual errors.

Note: Boosting is not parallelizable as training depends on previous predictors.

In [8]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200,
    learning_rate=0.5,
    random_state=42
)
ada_clf.fit(X_train, y_train)
print(f"AdaBoost Accuracy: {accuracy_score(y_test, ada_clf.predict(X_test)):.3f}")

AdaBoost Accuracy: 0.897


## 10. Stacking

Stacking trains a meta-model ("blender") to combine base models' predictions.
- Base models trained on part of the training set
- Meta-model trained on their predictions on a hold-out set
- More flexible than simple voting

Not directly supported in sklearn, but open-source libs like DESlib exist.

---

This notebook provides a hands-on overview and implementation guidance for ensemble methods and Random Forests based on Chapter 7.